In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
from  ecg_get_data import *
import neurokit2
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import random

import time
random_seed = 1
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

In [2]:

data_path =  '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/npy_ECG/' #路径
lable_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/label.npy'
model_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/model/'+time_str
os.makedirs(model_path, exist_ok=True)
log_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/log/'+  time_str


EcgChannles_num = 12
EcgLength_num = 5000

In [3]:
x = load_data(data_path,EcgChannles_num=EcgChannles_num,EcgLength_num=EcgLength_num)
y = load_label(lable_path)

100%|██████████| 1698/1698 [00:10<00:00, 164.56it/s]


In [4]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y,train_size=0.9,random_state = random_seed,shuffle = True,stratify=y)
print("         HTN     NHTN ")
print("train: %5d   %5d" % (train_y.sum(),len(train_y)-train_y.sum()))
print("test : %5d   %5d" % (test_y.sum(),len(test_y)-test_y.sum()))

         HTN     NHTN 
train:   628     900
test :    70     100


In [5]:
#train_dataset,valid_dataset = load_numpy_dataset_to_tensor_dataset(x,y,random_seed=random_seed,train_rate = 0.8)

In [6]:
BATCH_SIZE = 128
FOLDS = 5
EPOCHS = 640  
PATIENCE = 64
LR = 0.00001
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [7]:
import Models 
from torch.utils.tensorboard import SummaryWriter   
writer = SummaryWriter(log_path)
NET = [Models.CNN() for i in range(FOLDS)]
from torchsummary import summary
print(summary(NET[0], (EcgChannles_num,EcgLength_num), device="cpu"))
writer.add_graph(NET[0],torch.rand([1,EcgChannles_num,EcgLength_num]))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1             [-1, 12, 5000]              24
            Conv1d-2             [-1, 32, 5000]           4,256
              ReLU-3             [-1, 32, 5000]               0
       BatchNorm1d-4             [-1, 32, 5000]              64
            Conv1d-5             [-1, 64, 5000]          22,592
              ReLU-6             [-1, 64, 5000]               0
         MaxPool1d-7             [-1, 64, 1250]               0
       BatchNorm1d-8             [-1, 64, 1250]             128
            Conv1d-9            [-1, 128, 1250]          24,704
             ReLU-10            [-1, 128, 1250]               0
      BatchNorm1d-11            [-1, 128, 1250]             256
           Conv1d-12            [-1, 256, 1250]          98,560
             ReLU-13            [-1, 256, 1250]               0
        MaxPool1d-14             [-1, 2

In [8]:
from train_test_validat import *
from torch.optim.lr_scheduler import CosineAnnealingLR
import time

for fold in range(FOLDS):
    
    train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS)
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    train_dataloader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR)  
    criterion = torch.nn.CrossEntropyLoss()   
    #等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 32)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        #scheduler.step() # 学习率迭代

        time_all = time.time()-start_time
        validate_loss,validate_acc = test_model(train_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            torch.save(NET[fold],  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold])):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

**************************************************The  1 fold**************************************************
validate form     0 to   305 , all nums is   305
- Epoch: 1 - Train_loss: 0.65891 - Train_acc: 0.59026 - Val_loss: 0.68444 - Val_acc: 0.595103 - T_Time: 3.411
当前学习率：0.000010
Find better model in Epoch 1, saving model.
Validation loss decreased (inf --> 0.684437).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.62103 - Train_acc: 0.66823 - Val_loss: 0.66467 - Val_acc: 0.643684 - T_Time: 1.110
当前学习率：0.000010
Find better model in Epoch 2, saving model.
Validation loss decreased (0.684437 --> 0.664673).  Saving model ...
                    --------------------------------------------------

- Epoch: 3 - Train_loss: 0.60366 - Train_acc: 0.68682 - Val_loss: 0.64262 - Val_acc: 0.662148 - T_Time: 1.117
当前学习率：0.000010
Find better model in Epoch 3, saving model.
Validation loss decreased (0.664673 --> 0.642620).  Sav

KeyboardInterrupt: 